# 데이터 블록 만드는 법 (1)

> fastai에서는 데이터를 정의하는 방법으로 DataBlock API를 제안합니다. 각 인자가 의미하는 내용과, 실제 Siamese 공식 튜토리얼에 이 내용이 어떻게 적용되는지를 살펴봅니다.
- author: "Chansung Park"
- toc: true
- image: images/datablock/siamese-block.png
- comments: true
- categories: [datablock, siamese, fastai]
- permalink: /datablock-siamese/
- badges: false
- search_exclude: true

In [18]:
#hide
!pip install fastai
!pip install nbdev

In [20]:
#hide
from fastai.vision.all import *
import nbdev

In [14]:
#hide
# path = untar_data(URLs.PETS)
files = ""

def label_func(fname):
    return re.match(r'^(.*)_\d+.jpg$', fname.name).groups()[0]

class ImageTuple(fastuple):
    @classmethod
    def create(cls, fns): return cls(tuple(PILImage.create(f) for f in fns))
    
    def show(self, ctx=None, **kwargs): 
        t1,t2 = self
        if not isinstance(t1, Tensor) or not isinstance(t2, Tensor) or t1.shape != t2.shape: return ctx
        line = t1.new_zeros(t1.shape[0], t1.shape[1], 10)
        return show_image(torch.cat([t1,line,t2], dim=2), ctx=ctx, **kwargs)

def ImageTupleBlock():
    return TransformBlock(type_tfms=ImageTuple.create, batch_tfms=IntToFloatTensor)

# splits = RandomSplitter()(files)
# splits_files = [files[splits[i]] for i in range(2)]
# splits_sets = mapped(set, splits_files)

def get_split(f):
    for i,s in enumerate(splits_sets):
        if f in s: return i
    raise ValueError(f'File {f} is not presented in any split.')

# splbl2files = [{l: [f for f in s if label_func(f) == l] for l in labels} for s in splits_sets]

def splitter(items): 
    def get_split_files(i): 
        return [j for j,(f1,f2,same) in enumerate(items) if get_split(f1)==i]

    return get_split_files(0),get_split_files(1)

def draw_other(f):
    same = random.random() < 0.5

    cls = label_func(f)
    split = get_split(f)

    if not same: 
        cls = random.choice(L(l for l in labels if l != cls)) 

    return random.choice(splbl2files[split][cls]), same

def get_tuples(files): 
    return [[f, *draw_other(f)] for f in files]

def get_x(t): 
    return t[:2]

def get_y(t): 
    return t[2]

In [23]:
nbdev.show_doc(DataBlock)

<h2 id="DataBlock" class="doc_header"><code>class</code> <code>DataBlock</code><a href="https://github.com/fastai/fastai/tree/master/fastai/data/block.py#L58" class="source_link" style="float:right">[source]</a></h2>

> <code>DataBlock</code>(**`blocks`**=*`None`*, **`dl_type`**=*`None`*, **`getters`**=*`None`*, **`n_inp`**=*`None`*, **`item_tfms`**=*`None`*, **`batch_tfms`**=*`None`*, **`get_items`**=*`None`*, **`splitter`**=*`None`*, **`get_y`**=*`None`*, **`get_x`**=*`None`*)

Generic container to quickly build `Datasets` and `DataLoaders`

DataBlock은 모델이 학습하는 데이터를 준비시키는 핵심 API 입니다.

중요한 사실은 DataBlock은 일종의 **템플릿** 이라는 것입니다.
실제로 데이터가 **주입되었을 때**, **'이런 이런 식으로 동작한다'** 를 정의하는 것이죠.

상기 DataBlock API의 원형을 말로 풀어서 설명하면 다음과 같습니다.

0. `dls = siamese.dataloaders(files)`

1. 0 에서, 학습될 모든 데이터를 입력 받으면, 이를 모델에 입력 가능한 형태를 결정합니다. (`get_items`)
   - 여기서 전자의 '학습될 모든 데이터'란, 아무런 조치도 취해지지 않은 것입니다.
   - 후자의 '모델에 입력 가능한 형태'란, 입력/출력(타깃/레이블) 형식의 `tuple`입니다.
     - 단 여기서는 완전한 `raw` 형식입니다.
       예) 입력: 이미지 파일 이름, 출력: 레이블 값 / 이름 (원-핫 인코딩되어 있지 않습니다)

2. 입력 가능한 형태의 데이터에서, 무엇을 입력으로 삼을지 결정합니다. (`get_x`)
3. 입력 가능한 형태의 데이터에서, 무엇을 레이블로 삼을지 결정합니다. (`get_y`)

4. `raw`한 형식의 '모델에 입력 가능한' 데이터를, 실제 모델에 입력 가능한 형식으로 바꿔줄 규칙을 지정합니다. (`blocks`)
   - `blocks`는 두 개 이상의 여러개로 구성도 가능합니다.
   - 단 이때, 어디서 부터 어디까지가 입력이고 출력인지를 명시해야 하므로 `n_inp` 라는 인자가 사용됩니다.
     예. `DataBlock(..., n_inp=?,...)` => `n_inp`은 단순히 입력/출력을 구분하기 위한 인덱스 입니다.

5. 각 데이터 하나에 대한 변형을 하고 싶다면, 그 변형 규칙을 지정합니다. (`item_tfms`)
6. 배치 단위의 데이터에 대한 변형을 하고 싶다면, 그 변형 규칙을 지정합니다. (`batch_tfms`)
   - 배치 단위의 데이터 변형은 배치 단위로 GPU 에서 수행됩니다.

In [15]:
siamese = DataBlock(
       blocks=(ImageTupleBlock, CategoryBlock), # tuple 형식으로, 두 개 이상도 가능합니다. 
    get_items=get_tuples,                       # 모든 데이터를 불러 들이는 함수를 지정합니다.
        get_x=get_x,                            # 불러와진 데이터에 대해서, 입력을 결정하는 함수를 지정합니다.
        get_y=get_y,                            # 불러와진 데이터에 대해서, 출력을 결정하는 함수를 지정합니다.
     splitter=splitter,                         # 학습/검증 데이터셋을 분리하는 함수를 지정합니다.
    item_tfms=Resize(224),                      # 아이템 단위의 변환
   batch_tfms=[Normalize.from_stats(*imagenet_stats)]   # 배치 단위의 변환
)

앞선 설명을 Siamese 데이터 블록에 대입해서 설명해 보자면

0. `siamese.dataloaders(files)` 에서 입력된 files는 단순히 Path 객체로 표현된 이미지 파일 경로의 목록 입니다.

1. 이미지 파일 경로의 목록이 `get_items`에 전달되고, 또 다른 이미지를 무작위로 하나 선택하고, 두 이미지가 같은지/다른지를 (타깃) 결정하여 <= 이 세개의 내용을 반환합니다.
   - 그 다음, 두 이미지의 레이블을 look-up 합니다. look-up 방식은 정하기 나름입니다.
   - 이 예제에서의 레이블 look-up 방식은 상위 Parent 폴더 이름입니다.

2. `get_items`가 반환한 것은 세 개의 요소가 담긴 tuple 입니다.
   이 중 처음 두 개(이미지 1, 이미지 2)를 입력으로서 추출합니다 (`get_x`)
3. 그리고 마지막 한 개(레이블)을 출력으로서 추출합니다 (`get_y`)

4. 실제 모델에 입력 가능한 형식으로 바꿔줍니다.
   - 입력은 `PILImage` 이미지 형식으로 준비되지만, 이후 `TensorImage` 형식으로 전환됩니다
     - 이 과정은 `ImageTupleBlock` 함수가 반환하는 `TransformBlock`을 보면 알 수 있습니다.
     - `TransformBlock`은 `type_tfms`로 `PILImage`를 제안하지만, 추가적으로 `batch_tfms`에서는 `TensorImage`로의 전환을 제안합니다.
     - 여기서 등장한 `batch_tfms`는 최종적으로 `DataBlock` 에서 정의하는 `batch_tfms`와 머지되어, 나중에 수행됩니다.

   - 출력은 준비된 레이블을 원-핫 인코딩된 텐서 형태로 전환됩니다.

4. 각 데이터 하나에 대한 변형으로, 이미지 크기 조절을 수행합니다.
   - 이미지 크기 조절은 `Resize` 라는 `Transform` 형 객체를 활용합니다.
   - 적용하는 `Transform`은 준비된 데이터의 튜플(이미지1, 이미지2, 레이블)에 모두 적용되지만, 내부적으로 자신이 적용 가능한 녀석이 아니면 그대로 종료됩니다
     : 따라서 이미지1, 이미지2에 대한 변환이 수행됩니다.

5. 배치단위로 데이터를 변환합니다. 
   - 4번과 마찬가지로 적용될 대상을 스스로 알아챕니다
   - `Normalize.from_stats(*imagenet_stats)`는 이미지넷 데이터의 평균, 표준편차 값을 기준으로 데이터를 정규화 함을 의미합니다.